In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandasql import sqldf
import time


In [ ]:
#function to pull in some data
def schedule_scraper(year):
    url = 'https://www.pro-football-reference.com/years/{}/games.htm'.format(year)
    time.sleep(5)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    headers = ['Day', 'Date', 'Time', 'Winner_tie', '-', 'Loser_tie', 'label','PtsW', 'PtsL', 
               'YdsW', 'TOW', 'YdsL', 'TOL'] 
    rows = soup.findAll('tr', class_ = lambda table_rows: table_rows != "thead")  
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))] 
    player_stats = player_stats[1:]
    df_sched = pd.DataFrame(player_stats, columns = headers)
    return df_sched

In [ ]:
#Pull data for our initial year 2010
df = schedule_scraper(2010)
df_ravens = pd.concat([df[df.Winner_tie == 'Baltimore Ravens'], 
                       df[df.Loser_tie == 'Baltimore Ravens']], 
                      axis = 'index').sort_values(by = ['Date'])
#split data into ravens and their opponents
df_ravens1 = sqldf('''select Date, 
                     case when Winner_tie = "Baltimore Ravens" then 1 else 0 end as Win,
                     case when Winner_tie = "Baltimore Ravens" then 0 else 1 end as Loss,
                     case when Winner_tie = "Baltimore Ravens" then PtsW else PtsL end as Ravens_points,
                     case when Winner_tie = "Baltimore Ravens" then PtsL else PtsW end as Opp_points,
                     case when Winner_tie = "Baltimore Ravens" then YdsW else YdsL end as Ravens_yds,
                     case when Winner_tie = "Baltimore Ravens" then YdsL else YdsW end as Opp_yds,
                     case when Winner_tie = "Baltimore Ravens" then TOW else TOL end as Ravens_TO,
                     case when Winner_tie = "Baltimore Ravens" then TOL else TOW end as Opp_TO
                     from df_ravens''')
#sum to get season wins and season losses
df_ravens_autor = sqldf('''select *, 
                     SUM(Win) OVER (ORDER BY Date) AS Season_wins, 
                     SUM(Loss) OVER (ORDER BY Date) AS Season_losses
                     from df_ravens1''')

#repeat for years 2011-2021 and concatinate them to our original dataframe
for i in range(2011, 2023):
    df = schedule_scraper(i)
    df_ravens = pd.concat([df[df.Winner_tie == 'Baltimore Ravens'], 
                           df[df.Loser_tie == 'Baltimore Ravens']], 
                          axis = 'index').sort_values(by = ['Date'])
    df_ravens1 = sqldf('''select Date, 
                         case when Winner_tie = "Baltimore Ravens" then 1 else 0 end as Win,
                         case when Winner_tie = "Baltimore Ravens" then 0 else 1 end as Loss,
                         case when Winner_tie = "Baltimore Ravens" then PtsW else PtsL end as Ravens_points,
                         case when Winner_tie = "Baltimore Ravens" then PtsL else PtsW end as Opp_points,
                         case when Winner_tie = "Baltimore Ravens" then YdsW else YdsL end as Ravens_yds,
                         case when Winner_tie = "Baltimore Ravens" then YdsL else YdsW end as Opp_yds,
                         case when Winner_tie = "Baltimore Ravens" then TOW else TOL end as Ravens_TO,
                         case when Winner_tie = "Baltimore Ravens" then TOL else TOW end as Opp_TO
                         from df_ravens''')
    df_ravens2 = sqldf('''select *, 
                         SUM(Win) OVER (ORDER BY Date) AS Season_wins, 
                         SUM(Loss) OVER (ORDER BY Date) AS Season_losses
                         from df_ravens1''')
    df_ravens_autor = pd.concat([df_ravens_autor, df_ravens2], axis = 'index')
df_ravens_autor

In [4]:
#query to add in win/loss streak variables
df_ravens_autor_final = sqldf('''select a.*, 
                     CASE WHEN Win = 1 THEN 
                         (SELECT 1 + COUNT(*) FROM df_ravens_autor b
                          WHERE a.Win = b.Win AND a.Date > b.Date
                          AND NOT EXISTS (SELECT * FROM df_ravens_autor c
                     WHERE b.Win <> c.Win and c.Date BETWEEN b.Date AND a.Date))
                     ELSE 0 END AS Streak_W,
                     CASE WHEN Loss = 1 THEN 
                         (SELECT 1 + COUNT(*) FROM df_ravens_autor b
                          WHERE a.Loss = b.Loss AND a.Date > b.Date
                          AND NOT EXISTS (SELECT * FROM df_ravens_autor c
                     WHERE b.Loss <> c.Loss and c.Date BETWEEN b.Date AND a.Date))
                     ELSE 0 END AS Streak_L
                     from df_ravens_autor a''')
df_ravens_autor_final

,Date,Win,Loss,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
0,2010-09-13,1,0,10,9,282,176,3,1,1,0,1,0
1,2010-09-19,0,1,10,15,259,253,4,0,1,1,0,1
2,2010-09-26,1,0,24,17,365,304,0,0,2,1,1,0
3,2010-10-03,1,0,17,14,320,210,2,1,3,1,2,0
4,2010-10-10,1,0,31,17,415,346,0,1,4,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2022-12-17,0,1,3,13,324,283,2,0,9,5,0,1
221,2022-12-24,1,0,17,9,299,327,0,1,10,5,1,0
222,2023-01-01,0,1,13,16,240,351,1,0,10,6,0,1
223,2023-01-08,0,1,16,27,386,257,4,1,10,7,0,2


In [5]:
df_ravens_autor_final.index = df_ravens_autor_final.Date
df_ravens_autor_final = df_ravens_autor_final.drop('Date', axis = 'columns')
df_ravens_autor_final = df_ravens_autor_final.astype(float)
df_ravens_autor_final

,Win,Loss,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
Date,,,,,,,,,,,,
2010-09-13,1.0,0.0,10.0,9.0,282.0,176.0,3.0,1.0,1.0,0.0,1.0,0.0
2010-09-19,0.0,1.0,10.0,15.0,259.0,253.0,4.0,0.0,1.0,1.0,0.0,1.0
2010-09-26,1.0,0.0,24.0,17.0,365.0,304.0,0.0,0.0,2.0,1.0,1.0,0.0
2010-10-03,1.0,0.0,17.0,14.0,320.0,210.0,2.0,1.0,3.0,1.0,2.0,0.0
2010-10-10,1.0,0.0,31.0,17.0,415.0,346.0,0.0,1.0,4.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-17,0.0,1.0,3.0,13.0,324.0,283.0,2.0,0.0,9.0,5.0,0.0,1.0
2022-12-24,1.0,0.0,17.0,9.0,299.0,327.0,0.0,1.0,10.0,5.0,1.0,0.0
2023-01-01,0.0,1.0,13.0,16.0,240.0,351.0,1.0,0.0,10.0,6.0,0.0,1.0


In [6]:
df_ravens_autor_final.iloc[0,:]

Win                1.0
Loss               0.0
Ravens_points     10.0
Opp_points         9.0
Ravens_yds       282.0
Opp_yds          176.0
Ravens_TO          3.0
Opp_TO             1.0
Season_wins        1.0
Season_losses      0.0
Streak_W           1.0
Streak_L           0.0
Name: 2010-09-13, dtype: float64

In [7]:
#drop the loss variable to assure we have a nonsingular dataframe (win and loss are clearly dependent)
df_ravens_autor_final = df_ravens_autor_final.drop(['Loss'], axis = 'columns')

In [8]:
%run C:\Users\cpilo\.spyder-py3\autosave\Time_Series_Class.py

In [9]:
var_model = time_series_model(df_ravens_autor_final, nobs = 3, max_lags = 16)

In [10]:
model = var_model.get_var_model()
model

C:\Users\cpilo\Hello\lib\site-packages\statsmodels\tsa\base\tsa_model.py:216: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


In [12]:
forecasted_series = var_model.get_var_forecasted_data()
forecasted_series

C:\Users\cpilo\Hello\lib\site-packages\statsmodels\tsa\base\tsa_model.py:216: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
222,-1.207717,-3.329199,100.805146,372.117965,940.851278,7.589874,0.164317,5.939601,1.470317,6.208015,4.739265
223,1.920524,82.753750,-25.124807,909.906442,-67.680432,-4.826254,-0.546449,11.741251,-2.988310,5.913042,-0.187897
224,-0.211183,-34.579556,-18.530919,-85.119457,-47.268272,3.213438,3.333216,11.105009,0.052649,-0.518161,3.071385


In [13]:
accuracy = var_model.get_accuracy_matrix()
accuracy

C:\Users\cpilo\Hello\lib\site-packages\statsmodels\tsa\base\tsa_model.py:216: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\cpilo\.spyder-py3\autosave\Time_Series_Class.py:139: RuntimeWarning: divide by zero encountered in double_scalars
  pe = (np.sum(forecast)-np.sum(actual))/np.sum(actual)
C:\Users\cpilo\Hello\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\cpilo\Hello\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\cpilo\.spyder-py3\autosave\Time_Series_Class.py:146: RuntimeWarning: divide by zero encountered in true_divide
  minmax = 1 - np.mean(mins/maxs)             # minmax
C:\Users\cpilo\Hello\lib\site-packages\numpy\lib\function_bas

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
mape,inf,2.820764,3.000997,1.047202,1.381946,3.134386,inf,0.230222,1.058423,inf,1.619003
me,0.167208,-0.385002,-3.283527,68.968317,-5.365809,-0.340981,0.317028,-0.404713,-7.488448,3.867632,0.540918
mae,1.113141,44.887502,59.820291,368.381288,398.599994,5.543189,1.347994,2.302220,7.488448,4.213073,1.999516
mpe,NaN,-0.039358,0.532551,0.224639,-0.261620,1.663343,inf,-0.040471,-1.058423,NaN,0.889704
rmse,1.315496,49.608993,62.497433,405.644098,421.289193,6.397959,1.618889,2.629312,7.819735,4.958898,2.501606
corr,NaN,0.020969,-0.975805,0.185822,0.979201,-0.999771,0.343542,NaN,-0.311161,NaN,-0.332771
minmax,inf,1.698946,1.514649,0.721556,1.030761,1.150808,1.082146,0.217949,1.058423,inf,0.635396


In [14]:
num_differences = var_model.get_number_of_differences()
num_differences

1

In [15]:
initial_vals = var_model.get_initial_values()
initial_vals

[array([  1.,  10.,   9., 282., 176.,   3.,   1.,   1.,   0.,   1.,   0.])]

In [16]:
stationary_series = var_model.get_stationary_series()
stationary_series  

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
Date,,,,,,,,,,,
2010-09-19,-1.0,0.0,6.0,-23.0,77.0,1.0,-1.0,0.0,1.0,-1.0,1.0
2010-09-26,1.0,14.0,2.0,106.0,51.0,-4.0,0.0,1.0,0.0,1.0,-1.0
2010-10-03,0.0,-7.0,-3.0,-45.0,-94.0,2.0,1.0,1.0,0.0,1.0,0.0
2010-10-10,0.0,14.0,3.0,95.0,136.0,-2.0,0.0,1.0,0.0,1.0,0.0
2010-10-17,-1.0,-11.0,6.0,-38.0,48.0,0.0,1.0,0.0,1.0,-3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-17,-1.0,-13.0,-1.0,15.0,-46.0,2.0,-3.0,0.0,1.0,-2.0,1.0
2022-12-24,1.0,14.0,-4.0,-25.0,44.0,-2.0,1.0,1.0,0.0,1.0,-1.0
2023-01-01,-1.0,-4.0,7.0,-59.0,24.0,1.0,-1.0,0.0,1.0,-1.0,1.0


In [17]:
#save the important info to data dictionary
import pickle
df = {"team": 'Ravens',
      "variables": df_ravens_autor_final.columns,
      "model": model,
      "forecast": forecasted_series,
      "accuracy": accuracy,
      "number of times differenced": num_differences,
      "initial series values": initial_vals,
      "stationary input series": stationary_series
     }
#pickle the model and save to a file path
file_path = r"C:\Users\cpilo\Downloads\ravens_pickle_file.pkl"
pickle.dump(df, open(file_path, 'wb'))

In [18]:
rav_model_stuff = pickle.load(open(r"C:\Users\cpilo\Downloads\ravens_pickle_file.pkl", 'rb'))
rav_model_stuff

{'team': 'Ravens',
 'variables': Index(['Win', 'Ravens_points', 'Opp_points', 'Ravens_yds', 'Opp_yds',
        'Ravens_TO', 'Opp_TO', 'Season_wins', 'Season_losses', 'Streak_W',
        'Streak_L'],
       dtype='object'),
 'model': <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper at 0x1a573ce1940>,
 'forecast':           Win  Ravens_points  Opp_points  Ravens_yds     Opp_yds  Ravens_TO  \
 222 -1.207717      -3.329199  100.805146  372.117965  940.851278   7.589874   
 223  1.920524      82.753750  -25.124807  909.906442  -67.680432  -4.826254   
 224 -0.211183     -34.579556  -18.530919  -85.119457  -47.268272   3.213438   
 
        Opp_TO  Season_wins  Season_losses  Streak_W  Streak_L  
 222  0.164317     5.939601       1.470317  6.208015  4.739265  
 223 -0.546449    11.741251      -2.988310  5.913042 -0.187897  
 224  3.333216    11.105009       0.052649 -0.518161  3.071385  ,
 'accuracy':              Win  Ravens_points  Opp_points  Ravens_yds     Opp_yds  \
 mape         

In [19]:
df_ravens_autor_final

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
Date,,,,,,,,,,,
2010-09-13,1.0,10.0,9.0,282.0,176.0,3.0,1.0,1.0,0.0,1.0,0.0
2010-09-19,0.0,10.0,15.0,259.0,253.0,4.0,0.0,1.0,1.0,0.0,1.0
2010-09-26,1.0,24.0,17.0,365.0,304.0,0.0,0.0,2.0,1.0,1.0,0.0
2010-10-03,1.0,17.0,14.0,320.0,210.0,2.0,1.0,3.0,1.0,2.0,0.0
2010-10-10,1.0,31.0,17.0,415.0,346.0,0.0,1.0,4.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-17,0.0,3.0,13.0,324.0,283.0,2.0,0.0,9.0,5.0,0.0,1.0
2022-12-24,1.0,17.0,9.0,299.0,327.0,0.0,1.0,10.0,5.0,1.0,0.0
2023-01-01,0.0,13.0,16.0,240.0,351.0,1.0,0.0,10.0,6.0,0.0,1.0


In [20]:
my_ravens_model = rav_model_stuff['model']
my_ravens_model

In [21]:
#pull 17 most recent ravens games
my_data = df_ravens_autor_final.iloc[-17:]
initial_values = my_data.iloc[:1].values
#difference our data and drop the first column
my_data = my_data.diff().dropna()
my_data

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
Date,,,,,,,,,,,
2022-09-25,1.0,-1.0,-16.0,-79.0,-100.0,2.0,2.0,1.0,0.0,1.0,-1.0
2022-10-02,-1.0,-17.0,-3.0,-98.0,-121.0,0.0,-2.0,0.0,1.0,-1.0,1.0
2022-10-09,1.0,-1.0,-6.0,29.0,-35.0,-1.0,-1.0,1.0,0.0,1.0,-1.0
2022-10-16,-1.0,1.0,7.0,81.0,-53.0,1.0,0.0,0.0,1.0,-1.0,1.0
2022-10-23,1.0,3.0,-4.0,-152.0,98.0,-1.0,1.0,1.0,0.0,1.0,-1.0
2022-10-27,0.0,4.0,2.0,199.0,13.0,-1.0,-1.0,1.0,0.0,1.0,0.0
2022-11-07,0.0,0.0,-9.0,-134.0,-106.0,0.0,0.0,1.0,0.0,1.0,0.0
2022-11-20,0.0,-14.0,-10.0,-11.0,-38.0,1.0,2.0,1.0,0.0,1.0,0.0
2022-11-27,-1.0,14.0,25.0,107.0,127.0,1.0,-2.0,0.0,1.0,-4.0,1.0


In [22]:
initial_values

array([[  0.,  38.,  42., 473., 547.,   0.,   2.,   1.,   1.,   0.,   1.]])

In [23]:
df_forecast = my_ravens_model.forecast(my_data.values, steps = 3)
df_forecast = pd.DataFrame(df_forecast, columns = my_data.columns)
df_forecast

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
0,0.326832,-15.808475,-16.221631,133.063608,40.272859,3.488012,-0.634884,-7.814422,-7.104835,0.405991,-3.482523
1,0.585201,32.375515,9.693726,-73.527592,-7.442176,-4.615212,1.430575,0.277348,0.095841,-0.266032,-1.077523
2,-0.357896,-55.428986,19.136438,-305.758254,129.319303,4.734454,-0.898456,1.562289,0.489078,3.582839,1.532583


In [24]:
my_data.reset_index(drop = True)

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
0,1.0,-1.0,-16.0,-79.0,-100.0,2.0,2.0,1.0,0.0,1.0,-1.0
1,-1.0,-17.0,-3.0,-98.0,-121.0,0.0,-2.0,0.0,1.0,-1.0,1.0
2,1.0,-1.0,-6.0,29.0,-35.0,-1.0,-1.0,1.0,0.0,1.0,-1.0
3,-1.0,1.0,7.0,81.0,-53.0,1.0,0.0,0.0,1.0,-1.0,1.0
4,1.0,3.0,-4.0,-152.0,98.0,-1.0,1.0,1.0,0.0,1.0,-1.0
5,0.0,4.0,2.0,199.0,13.0,-1.0,-1.0,1.0,0.0,1.0,0.0
6,0.0,0.0,-9.0,-134.0,-106.0,0.0,0.0,1.0,0.0,1.0,0.0
7,0.0,-14.0,-10.0,-11.0,-38.0,1.0,2.0,1.0,0.0,1.0,0.0
8,-1.0,14.0,25.0,107.0,127.0,1.0,-2.0,0.0,1.0,-4.0,1.0
9,1.0,-17.0,-19.0,-130.0,-60.0,0.0,-1.0,1.0,0.0,1.0,-1.0


In [25]:
df_result = pd.concat([my_data, df_forecast], axis = 'index').reset_index(drop = True)
df_result

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
0,1.000000,-1.000000,-16.000000,-79.000000,-100.000000,2.000000,2.000000,1.000000,0.000000,1.000000,-1.000000
1,-1.000000,-17.000000,-3.000000,-98.000000,-121.000000,0.000000,-2.000000,0.000000,1.000000,-1.000000,1.000000
2,1.000000,-1.000000,-6.000000,29.000000,-35.000000,-1.000000,-1.000000,1.000000,0.000000,1.000000,-1.000000
3,-1.000000,1.000000,7.000000,81.000000,-53.000000,1.000000,0.000000,0.000000,1.000000,-1.000000,1.000000
4,1.000000,3.000000,-4.000000,-152.000000,98.000000,-1.000000,1.000000,1.000000,0.000000,1.000000,-1.000000
5,0.000000,4.000000,2.000000,199.000000,13.000000,-1.000000,-1.000000,1.000000,0.000000,1.000000,0.000000
6,0.000000,0.000000,-9.000000,-134.000000,-106.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
7,0.000000,-14.000000,-10.000000,-11.000000,-38.000000,1.000000,2.000000,1.000000,0.000000,1.000000,0.000000
8,-1.000000,14.000000,25.000000,107.000000,127.000000,1.000000,-2.000000,0.000000,1.000000,-4.000000,1.000000
9,1.000000,-17.000000,-19.000000,-130.000000,-60.000000,0.000000,-1.000000,1.000000,0.000000,1.000000,-1.000000


In [27]:
#use the invert_differencing function from our time series object to get our actual forecast
undifferenced_forecast = var_model.invert_differencing(initial_values[0], df_result)
undifferenced_forecast

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
0,0.000000,38.000000,42.000000,473.000000,547.000000,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000
1,1.000000,37.000000,26.000000,394.000000,447.000000,2.000000,4.000000,2.000000,1.000000,1.000000,0.000000
2,0.000000,20.000000,23.000000,296.000000,326.000000,2.000000,2.000000,2.000000,2.000000,0.000000,1.000000
3,1.000000,19.000000,17.000000,325.000000,291.000000,1.000000,1.000000,3.000000,2.000000,1.000000,0.000000
4,0.000000,20.000000,24.000000,406.000000,238.000000,2.000000,1.000000,3.000000,3.000000,0.000000,1.000000
5,1.000000,23.000000,20.000000,254.000000,336.000000,1.000000,2.000000,4.000000,3.000000,1.000000,0.000000
6,1.000000,27.000000,22.000000,453.000000,349.000000,0.000000,1.000000,5.000000,3.000000,2.000000,0.000000
7,1.000000,27.000000,13.000000,319.000000,243.000000,0.000000,1.000000,6.000000,3.000000,3.000000,0.000000
8,1.000000,13.000000,3.000000,308.000000,205.000000,1.000000,3.000000,7.000000,3.000000,4.000000,0.000000
9,0.000000,27.000000,28.000000,415.000000,332.000000,2.000000,1.000000,7.000000,4.000000,0.000000,1.000000


In [28]:
#Our actual forecasted values are the last three rows of this dataframe
#note they are different from our forecasted data we saved earlier because we didn't split the data
#into training and testing sets this time
undifferenced_forecast.iloc[-3:]

,Win,Ravens_points,Opp_points,Ravens_yds,Opp_yds,Ravens_TO,Opp_TO,Season_wins,Season_losses,Streak_W,Streak_L
17,0.326832,1.191525,7.778369,497.063608,274.272859,5.488012,0.365116,2.185578,0.895165,0.405991,-0.482523
18,0.912033,33.567040,17.472095,423.536016,266.830682,0.872800,1.795690,2.462927,0.991006,0.139960,-1.560046
19,0.554137,-21.861945,36.608533,117.777762,396.149986,5.607254,0.897234,4.025216,1.480084,3.722799,-0.027463
